# Setup Visdom

Install it with:

`pip install visdom`

Start the server:

`python -m visdom.server`

Visdom now can be accessed at http://localhost:8097 in the browser.


# LDA Training Visualization

To monitor the LDA training, a list of Metrics can be passed to the LDA function call for plotting their values live as the training progresses.  

Let's plot the training stats for an LDA model being trained on Lee corpus. We will use the four evaluation metrics available for topic models in gensim: Coherence, Perplexity, Topic diff and Convergence. (using separate hold_out and test corpus for evaluating the perplexity)

In [2]:
import os
import re
import gensim
from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary

# Set file names for train and test data
test_data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_train_file = test_data_dir + os.sep + 'lee_background.cor'
lee_test_file = test_data_dir + os.sep + 'lee.cor'

def read_corpus(fname):
    texts = []
    with open(fname, encoding="ISO-8859-1") as f:
        for line in f:
            # lower case all words
            lowered = line.lower()
            # remove punctuation and split into seperate words
            words = re.compile('\w+').findall(lowered)
            texts.append(words)
    return texts

training_texts = read_corpus(lee_train_file)
test_texts = read_corpus(lee_test_file)

# Split test data into hold_out and test corpus
holdout_texts = test_texts[:150]
test_texts = test_texts[150:]

dictionary = Dictionary(training_texts)

training_corpus = [dictionary.doc2bow(text) for text in training_texts]
holdout_corpus = [dictionary.doc2bow(text) for text in holdout_texts]
test_corpus = [dictionary.doc2bow(text) for text in test_texts]

In [ ]:
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric

# define perplexity callback for hold_out and test corpus
pl_holdout = PerplexityMetric(corpus=holdout_corpus, logger="visdom", viz_env="LdaModel", title="Perplexity")
pl_test = PerplexityMetric(corpus=test_corpus, logger="visdom", viz_env="LdaModel", title="Perplexity")

# define other remaining metrics available
ch_umass = CoherenceMetric(corpus=training_corpus, coherence="u_mass", logger="visdom", viz_env="LdaModel", title="Coherence (u_mass)")
diff_kl = DiffMetric(distance="kullback_leibler", logger="visdom", viz_env="LdaModel", title="Diff (kullback_leibler)")
convergence_jc = ConvergenceMetric(distance="jaccard", logger="visdom", viz_env="LdaModel", title="Convergence (jaccard)")

callbacks = [pl_holdout, pl_test, ch_umass, diff_kl, convergence_jc]

# training LDA model
model = ldamodel.LdaModel(corpus=training_corpus, id2word=dictionary, passes=5, num_topics=5, callbacks=callbacks)

When the model is set for training, you can open http://localhost:8097 to see the training progress.

In [ ]:
# to get a metric value on a trained model
print(CoherenceMetric(corpus=training_corpus, coherence="u_mass").get_value(model=model))

The four types of graphs which are plotted for LDA:

**Coherence**

Coherence is a measure used to evaluate topic models. A good model will generate coherent topics, i.e., topics         with high topic coherence scores. Good topics are topics that can be described by a short label based on the topic     terms they spit out. 

<img src="Coherence.gif">

Now, this graph along with the others explained below, can be used to decide if it's time to stop the training. We     can see if the value stops changing after some epochs and that we are able to get the highest possible coherence       of our model.  


**Perplexity**

Perplexity is a measurement of how well a probability distribution or probability model predicts a sample. In LDA, topics are described by a probability distribution over vocabulary words. So, perplexity can be used to compare probabilistic models like LDA.

<img src="Perplexity.gif">

For a good model, perplexity should be as low as possible.


**Topic Difference**

Topic Diff calculates the distance between two LDA models. This distance is calculated based on the topics, by either using their probability distribution over vocabulary words (kullback_leibler, hellinger) or by simply using the common vocabulary words between the topics from both model.

<img src="Diff.gif">

In the heatmap, X-axis define the Epoch no. and Y-axis define the distance between the identical topic from consecutive epochs. For ex. a particular cell in the heatmap with values (x=3, y=5, z=0.4) represent the distance(=0.4) between the topic 5 from 3rd epoch and topic 5 from 2nd epoch. With increasing epochs, the distance between the identical topics should decrease.
  
  
**Convergence**

Convergence is the sum of the difference between all the identical topics from two consecutive epochs. It is basically the sum of column values in the heatmap above.

<img src="Convergence.gif">

The model is said to be converged when the convergence value stops descending with increasing epochs.